In [1]:
from tkinter import *
class Page(object):
    def __init__(self):
        pass
#build a input page    
    def InputPage(self,root):
        root.title('Finance SuperMan')
        #root.geometry('300*100')
        self.user_input=StringVar()
        self.user_ouput=StringVar()
        
        #self.stoklist=self.read_stocklist
        #CREATE ATTRIBUTES FOR THIS INTERFACE
        self.title_1=Label(root,text="Welcome to Finance SuperMAN!")
        self.entry_label=Label(root,text="Enter a company: ")
        self.entry_box=Entry(root,textvariable=self.user_input)
        self.confirmButton=Button(root,text="confirm",
                                  command=self.OutputPage)
        #LAYOUTS THE WIDGETS
        self.title_1.grid(row=0,sticky=W)
        self.entry_label.grid(row=3,column=0,sticky=W)
        self.entry_box.grid(row=3,column=3,columnspan=2)
        self.confirmButton.grid(row=16,column=4,sticky=E)
    
#build a outputpage        
    def OutputPage(self):
        from tkinter import messagebox
        output=Toplevel()
        output.title('in-time Information')
        output.geometry('300x1000')
        title_1=Label(output,text="Welcome to Finance SuperMAN!")
        title_1.grid(row=0,sticky=W)
        ref=self.read_stocklist()
        ipt=self.user_input.get().upper()
        if ipt not in ref:
            messagebox.showinfo('enter error','please enter the right ticket name')
        else:
            #option button
            Button_competitor=Button(output,text="review competitor",
                                     command=lambda:self.get_competitor(ipt))
            Button_competitor.grid(row=5,column=0)
#         Button_competitor=Button(output,text="review competitor",
#                                   command=lambda:self.get_competitor(self.user_input.get().upper()))
#         Button_competitor.grid(row=5,column=0)
    
#validate input 
#     def validate_input(self,ipt=None):
#         from tkinter import messagebox
#         ref=self.read_stocklist()
#         ipt=ipt.get()
#         print(ipt)
#         print(ref)
#         if ipt not in ref and ipt!=None:       
#             messagebox.showinfo("Enter Error", "Please enter the right ticket name")
#         else:
#             self.OutputPage
#read in the company list        
    def read_stocklist(self):
        import pandas as pd
        company=pd.DataFrame()
        company=pd.read_csv('companylist.csv',usecols=['Symbol'])
        stocklist=company.append(pd.read_csv('companylist-2.csv',usecols=['Symbol'],header=0))
        stocklist=company['Symbol'].tolist()
        return stocklist
                                 
#Output: get competitor
    def get_competitor(self,stock):
        import requests
        from bs4 import BeautifulSoup
        url = 'https://www.nasdaq.com/symbol/' + stock + '/stock-comparison'

        response = requests.get(url)
        results_page = BeautifulSoup(response.content, 'lxml')
        table = results_page.find('div', class_='genTable thin')
        competitors = list()
        for i in range(4):
            competitors.append(table.find_all('input')[i].get('value'))
        print(competitors)   

    
    
    
    
root=Tk()
app=Page()
app.InputPage(root)
root.mainloop()

['GOOG', 'FB', 'GOOGL', 'RHT']
